# imports

In [6]:
from google.cloud import bigquery
import os
from google.oauth2 import service_account
import cloudvolume
from cloudvolume.exceptions import EmptyFileException

# credentials

## google web auth

In [17]:
'''
conda install conda-forge::google-cloud-sdk

Then, launch jupyter lab 

In a code cell, run bash command <!gcloud auth login > (https://cloud.google.com/sdk/gcloud/reference/auth/login)
    a browser tab should open up

RESULT:
You are now logged in as [kperky@gmail.com].
Your current project is [lcht-goog-connectomics].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
'''
!gcloud auth login 

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=dt3acp0ZGzufwwpQTBKhqXz1FL4nlb&access_type=offline&code_challenge=GrPafJbuLy1F16fANz5QjHWVitZhbAkj3DMa-3uomrI&code_challenge_method=S256


You are now logged in as [kperky@gmail.com].
Your current project is [lcht-goog-connectomics].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


## with key path (service account?)

In [16]:
key_path = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/keys/krista_credentials.json'

In [17]:
#Set environment variables for your notebook
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_path#'C:/Users/mpetkova/Dropbox/CREST/mariela_fish_credentials.json'
cred_str = os.environ['GOOGLE_APPLICATION_CREDENTIALS']
print(f'using credential file: {cred_str}')

using credential file: /Users/kperks/Documents/ell-connectome/eCREST-local-files/keys/krista_credentials.json


# load instance cloudvolume and bigquery client

In [18]:
# I get the error for both of these ways of initializing a client

bigquery_client = bigquery.Client(project='lcht-goog-connectomics')
# bigquery_client = bigquery.Client.from_service_account_json(json_credentials_path=key_path)

In [2]:
efish_cloudvolume = cloudvolume.CloudVolume('gs://fish-ell/roi450um_seg32fb16fb_220930', progress=False)

# do query

In [8]:
all_verts = []
for segment_id in ['299392199','299405462','299439953','299496636','386224676']:
    try:
      skel = efish_cloudvolume.skeleton.get(int(segment_id));
      # Initialize a NetworkX graph
      # Iterate over each vertex in the skeleton
      for v in skel.vertices:
          
          #zero y from molecular layer plane
          # yoffset = func((v[0], v[2]), *popt)
          y_adj = v[1] #- yoffset
          v = [int(v[0]),int(y_adj),int(v[2])]
    
          # Append the vertex coordinates to the list
          all_verts.append(list(v))
    except EmptyFileException:
        print(f'no skeleton for {segment_id}')
        continue

In [9]:
all_verts

[[246880, 260496, 55710],
 [246800, 260576, 55740],
 [246592, 260784, 55860],
 [246496, 260880, 55800],
 [246416, 260960, 55770],
 [246272, 261120, 56010],
 [246224, 261168, 56100],
 [246064, 261008, 56310],
 [246048, 260992, 56340],
 [246048, 261008, 56370],
 [246128, 261136, 56640],
 [246256, 261072, 56910],
 [246368, 260784, 56880],
 [246544, 260752, 57150],
 [246624, 260880, 57390],
 [246784, 261136, 57450],
 [247008, 261296, 57600],
 [247088, 261296, 57660],
 [247264, 261472, 57690],
 [247296, 261504, 57690],
 [247472, 261664, 57720],
 [247584, 261936, 57840],
 [247648, 262176, 58020],
 [247648, 262208, 58080],
 [247792, 262320, 58140],
 [248064, 262176, 58140],
 [248368, 262176, 58140],
 [248624, 262016, 58200],
 [248784, 261856, 58260],
 [248608, 261648, 58110],
 [248736, 261376, 58110],
 [248720, 261072, 58140],
 [248720, 260768, 58110],
 [248736, 260736, 58110],
 [248960, 260592, 58200],
 [249264, 260624, 58260],
 [249520, 260608, 58380],
 [249744, 260528, 58320],
 [250000, 26

In [19]:
base_seg_list = ['299392199','299405462','299439953','299496636','386224676'] # just some base seg IDs to try it with
base_seg_str = ', '.join(str(x) for x in base_seg_list)
    

QUERY = """
SELECT
    cast(objects.id as INT64) as seg_id,
    sample_voxel.x as x,
    sample_voxel.y as y,
    sample_voxel.z as z,
FROM
    `lcht-goog-connectomics.ell_roi450um_seg32fb16fb_220930.objinfo` as objects
WHERE objects.id in {}
""".format('('+base_seg_str+')')

df = bigquery_client.query(QUERY).to_dataframe()

RefreshError: ('invalid_grant: Invalid JWT Signature.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT Signature.'})